<a href="https://colab.research.google.com/github/sydney-machine-learning/sentimentanalysis-COVID19news/blob/main/Roberta_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# settings

In [1]:
!pip install transformers
!pip install scipy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
SEED = 1024
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

from torch.utils.data import TensorDataset, DataLoader, Dataset, RandomSampler, SequentialSampler
import torchtext
from torchtext import data

In [4]:
# Mount Google Drive
drive.mount('/content/drive')
# Load the RoBERTa tokenizer
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)
# model.to(device)
# Read the CSV file into a DataFrame
file_path = "/content/drive/MyDrive/Colab Notebooks/labeledEn.csv"
df = pd.read_csv(file_path)

# Check the shape of data
print(df.shape)

# Print the first few rows of the DataFrame
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(10000, 12)
             ID                                              Tweet  \
0  1.245140e+18  A glass of wine keeps the corona away- DRAKE. ...   
1  1.245140e+18  Can Anyone tell me if you took the flu shot la...   
2  1.245140e+18  Btw producers send me beats Im working on musi...   
3  1.245140e+18  When someone you know.. apart of your family d...   
4  1.245140e+18  Dear soccer, I really miss you ,please come ba...   

   Optimistic  Thankful  Empathetic  Pessimistic  Anxious  Sad  Annoyed  \
0           1         0           0            0        0    0        0   
1           0         0           0            0        1    0        0   
2           1         0           0            0        0    0        0   
3           0         0           0            0        0    1        0   
4           0         0           0            0        0    1 

# preprocessing1

In [5]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [6]:
contractionsWithAnotherInvertedComma = {
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [7]:
!pip install demoji

In [8]:
import string
import nltk
import csv
import re
import demoji
from nltk.stem.wordnet import WordNetLemmatizer

# prepro_func

In [9]:
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions

    def lower(self, tweet):
        return tweet.lower()

    def abbreviate(self, tweet):
        tweet = tweet.split(' ')
        j = 0
        for str_ in tweet:
            fileName = '/content/drive/MyDrive/Colab Notebooks/Abbreviations.txt'
            accessMode = 'r'
            with open(fileName, accessMode) as csvfile:
                dataFromFile = csv.reader(csvfile, delimiter = '=')
                str_ = re.sub('[^a-zA-Z0-9-_.]', '', str_)
                for row in dataFromFile:
                    if str_.upper() == row[0]:
                        tweet[j] = row[1]
                csvfile.close()
            j += 1
        return ' '.join(tweet)

    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet

    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])

        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)

    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)

    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)
        return tweet

In [10]:
def wordReplace(tweet):
    tweet = tweet.replace("indiavscorona", "india versus coronavirus")
    tweet = tweet.replace("outbreakindia", "outbreak india")
    tweet = tweet.replace("real”", "real")
    tweet = tweet.replace("mutra", "urine")
    tweet = tweet.replace("fakenews", "fake news")
    tweet = tweet.replace("“omg", "oh my god")
    tweet = tweet.replace("“damn", "damn")
    tweet = tweet.replace("god’s", "gods")
    tweet = tweet.replace("lockdownextension", "lockdown extension")
    tweet = tweet.replace("कोरोना", "coronavirus")
    tweet = tweet.replace("indiathanks", "india thanks")
    tweet = tweet.replace("coronacoronavirus", "coronavirus")
    tweet = tweet.replace('coronavirusinsa', "coronavirus in south africa")
    tweet = tweet.replace('coronaviruscanada', 'coronavirus canada')
    tweet = tweet.replace('coronavirusau', 'coronavirus australia')
    tweet = tweet.replace('coronavirusaus', 'coronavirus australia')
    tweet = tweet.replace('cuomoprimetime', 'new york governor prime time')
    tweet = tweet.replace('letsfightcoronavirus', 'let us fight coronavirus')
    tweet = tweet.replace("covid19", "coronavirus")
    tweet = tweet.replace("covid", "coronavirus")
    tweet = tweet.replace("aprilfoolsday", "april fools day")
    tweet = tweet.replace("covidー19", "coronavirus")
    tweet = tweet.replace("stayathome", "stay at home")
    tweet = tweet.replace("“april", "april")
    tweet = tweet.replace("“i", "i")
    tweet = tweet.replace("aprilfools", "april fools")
    tweet = tweet.replace("coronavirusoutbreak", "coronavirus outbreak")
    tweet = tweet.replace("virusー19", "coronavirus")
    tweet = tweet.replace("fool’s", "fools")
    tweet = tweet.replace("what’s", "what is")
    tweet = tweet.replace("coronavirus”", "coronavirus")
    tweet = tweet.replace("fools”", "fools")
    tweet = tweet.replace("stayhome", "stay home")
    tweet = tweet.replace("quarantinelife", "quarantine life")
    tweet = tweet.replace("tablighijamaat", "muslims")
    tweet = tweet.replace("corona”", "coronavirus")
    tweet = tweet.replace("fauci", "physician")
    tweet = tweet.replace("april’s", "april")
    tweet = tweet.replace("pmkcallscurfewextension", "prime minister calls for curfew extension")
    tweet = tweet.replace("“virus", "coronavirus")
    tweet = tweet.replace("virus”", "coronavirus")
    tweet = tweet.replace("“corona", "coronavirus")
    tweet = tweet.replace("coronavirustruth", "coronavirus truth")
    tweet = tweet.replace("socialdistancing", "social distancing")
    tweet = tweet.replace("homestaysafe", "home stay safe")
    tweet = tweet.replace("“coronavirus", "coronavirus")
    tweet = tweet.replace("coronavirusupdate", "coronavirus update")
    tweet = tweet.replace("virusvirus", "coronavirus")
    tweet = tweet.replace("coronaviruspandemic", "coronavirus pandemic")
    tweet = tweet.replace("thelockdown", "the lockdown")
    tweet = tweet.replace("nizamuddin", "delhi")
    tweet = tweet.replace("trump’s", "donald trump")
    tweet = tweet.replace("“the", "the")
    tweet = tweet.replace("virus2019", "coronavirus")
    tweet = tweet.replace("indiafightscorona", "india fights coronavirus")
    tweet = tweet.replace("homesavelives", "home save lives")
    tweet = tweet.replace("everyone’s", "everyone")
    tweet = tweet.replace("coronariskforprisoners", "coronavirus risk for prisoners")
    tweet = tweet.replace("coronavirususa", "coronavirus usa")
    tweet = tweet.replace("tablighi", "mosque")
    tweet = tweet.replace("delhimarkaz", "delhi mosque")
    tweet = tweet.replace("coronajihad", "coronavirus struggle")
    tweet = tweet.replace("coronajihaad", "coronavirus struggle")
    tweet = tweet.replace("aprilfool", "april fool")
    tweet = tweet.replace("trumppressconference", "trump press conference")
    tweet = tweet.replace("i’m", "i am")
    tweet = tweet.replace("tigerking", "tiger king")
    tweet = tweet.replace("it’s", "it is")
    tweet = tweet.replace("trumpvirus", "trump virus")
    tweet = tweet.replace("today’s", "today is")
    tweet = tweet.replace("“you", "you")
    tweet = tweet.replace("“a", "a")
    tweet = tweet.replace("fools’", "fools")
    tweet = tweet.replace("rtgnews", "news")
    tweet = tweet.replace("19india", "india")
    tweet = tweet.replace("coronavirusindia", "coronavirus india")
    tweet = tweet.replace("y’all", "you all")
    tweet = tweet.replace("मीडिया", "media")
    tweet = tweet.replace("here’s", "here is")
    tweet = tweet.replace("“we", "we")
    tweet = tweet.replace("“fuck", "fuck")
    tweet = tweet.replace("flattenthecurve", "flatten the curve")
    tweet = tweet.replace("jammuandkashmir", "jammu and kashmir")
    tweet = tweet.replace("chriscuomo", "new york governor")
    tweet = tweet.replace("‘april", "april")
    tweet = tweet.replace("dranbumani", "doctor")
    tweet = tweet.replace("tndemandsmasstesting", "tamil nadu demands mass testing")
    tweet = tweet.replace("tabligi", "muslims")
    tweet = tweet.replace("don’t", "do not")
    tweet = tweet.replace("वायरस", "virus")
    tweet = tweet.replace("letsfightvirus", "let us fight virus")
    return tweet

# prepro2

In [11]:
pp_class = preprocess(df, contractions, contractionsWithAnotherInvertedComma)
df['Tweet'] = df['Tweet'].apply(lambda x : pp_class.preprocess_tweet(x))

In [12]:
df['Tweet'] = df['Tweet'].str.lower()
df.head(10)

,ID,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,1.245140e+18,a glass of wine keeps the corona away drake ...,1,0,0,0,0,0,0,0,0,1
1,1.245140e+18,can anyone tell me if you took the flu shot la...,0,0,0,0,1,0,0,0,0,0
2,1.245140e+18,by the way producers send me beats im working ...,1,0,0,0,0,0,0,0,0,1
3,1.245140e+18,when someone you know apart of your family d...,0,0,0,0,0,1,0,0,0,0
4,1.245140e+18,dear soccer i really miss you please come ba...,0,0,0,0,0,1,1,0,0,0
5,1.245140e+18,new home remedy to treat coronavirus tested b...,1,0,0,0,0,0,0,1,0,1
6,1.245140e+18,when xavier wulf does an attack on titan tape ...,0,0,0,0,0,0,0,0,0,1
7,1.245140e+18,mouthwash is hand san for your mouth and i do ...,0,0,0,0,0,1,0,0,0,1
8,1.245140e+18,yes all of them n france 1 000 christians to...,0,0,0,0,0,0,1,0,1,0
9,1.245140e+18,update i destroyed the tire honestly if i get ...,0,0,0,1,0,1,0,0,0,0


In [13]:
def check_coverage(vocab, embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:
            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key = operator.itemgetter(1))[::-1]

    return sorted_x

def build_vocab(sentences, verbose = True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except:
                vocab[word] = 1
    return vocab

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path, encoding = "utf-8") as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)


def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [14]:
# Load the pre-trained GloVe word embeddings model file from the specified path and store it as a dictionary-formatted embedding matrix.
GLOVE_EMBEDDING_FILE = '/content/drive/MyDrive/Colab Notebooks/glove.840B.300d.txt'
glove_embeddings = load_embeddings(GLOVE_EMBEDDING_FILE)

# Print the number of loaded word vectors.
print(f'loaded {len(glove_embeddings)} word vectors ')

loaded 2196007 word vectors 


In [15]:
!pip install tqdm
from tqdm import tqdm
import operator

In [16]:
vocab = build_vocab(list(df['Tweet'].apply(lambda x : x.split())))
oov = check_coverage(vocab, glove_embeddings)
oov[:10]

100%|██████████| 14376/14376 [00:00<00:00, 633401.76it/s]

Found embeddings for 86.61% of vocab
Found embeddings for  96.72% of all text


[('covid19', 2240),
 ('covid', 408),
 ('aprilfoolsday', 135),
 ('stayathome', 73),
 ('aprilfools', 48),
 ('coronavirusoutbreak', 34),
 ('stayhome', 29),
 ('quarantinelife', 26),
 ('tablighijamaat', 25),
 ('fauci', 22)]

In [17]:
df['Tweet'] = df['Tweet'].apply(lambda x : wordReplace(x))

# build model

In [18]:
from sklearn.model_selection import train_test_split
sen_train, sen_test = train_test_split(df, train_size = 0.9, random_state = 1024)

sen_train.to_csv("train.csv", index = False)
sen_test.to_csv("test.csv", index = False)
sen_train.head(1)

,ID,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
560,1.245140e+18,minister for agriculture mahendra reddy to hol...,0,0,0,0,0,0,0,0,1,0


In [19]:
# !pip install torchtext
!pip install torchtext==0.6.0

In [20]:
import spacy
import torch
from torchtext.data import TabularDataset, Field
import re

nlp = spacy.load("en_core_web_sm")

def tokenizer(tweet):
    tweet = re.sub(r'[\n]', ' ', tweet)
    return [tok.text for tok in nlp(tweet)]

TWEET = Field(sequential=True, lower=True, tokenize=tokenizer)
LABEL = Field(sequential=False, use_vocab=False)

dataFields = [("ID", None), ("Tweet", TWEET), ("Optimistic", LABEL), ("Thankful", LABEL),
              ("Empathetic", LABEL), ("Pessimistic", LABEL), ("Anxious", LABEL), ("Sad", LABEL),
              ("Annoyed", LABEL), ("Denial", LABEL), ("Official report", LABEL), ("Joking", LABEL)]

train_dataset, test_dataset = TabularDataset.splits(
    path='/content/', train='train.csv', test='test.csv', format='csv', fields=dataFields, skip_header=True
)

In [21]:
print("Number of training samples : {}\n Number of testing samples : {}".format(len(train_dataset), len(test_dataset)))

Number of training samples : 9000
 Number of testing samples : 1000


In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
TWEET.build_vocab(train_dataset, vectors = 'glove.840B.300d')

.vector_cache/glove.840B.300d.zip: 2.18GB [06:49, 5.31MB/s]                            
100%|█████████▉| 2196016/2196017 [06:36<00:00, 5544.13it/s]


# Using RoBERTa

In [23]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [24]:
# Drop the 'ID' column from the DataFrame
df1 = df.drop(['ID'], axis=1)
# Create a new column named 'list' containing values of columns 1 to 10 as lists
df1['list'] = df1[df1.columns[1:11]].values.tolist()

# Create a new DataFrame 'new_df' containing the 'Tweet' column and the newly created 'list' column
new_df = df1[['Tweet', 'list']].copy()
new_df.head()


,Tweet,list
0,a glass of wine keeps the corona away drake ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,can anyone tell me if you took the flu shot la...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
2,by the way producers send me beats im working ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,when someone you know apart of your family d...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
4,dear soccer i really miss you please come ba...,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]"


In [25]:
MAX_LEN = 200  # Based on the length of tweets
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 1 # 4
LEARNING_RATE = 2e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')  # Use RoBERTa tokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [26]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.tweet = dataframe['Tweet']
        self.targets = self.dataframe.list
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, index):
        tweet = str(self.tweet[index])
        tweet = " ".join(tweet.split())

        inputs = self.tokenizer.encode_plus(
            tweet,
            None,
            add_special_tokens=True,  # Add special tokens for RoBERTa
            max_length=self.max_len,
            padding='max_length',  # Pad to max_length
            return_token_type_ids=True,
            return_tensors='pt',  # Return PyTorch tensors
            truncation=True  # Truncate sequences longer than max_length
        )
        input_ids = inputs['input_ids'].squeeze(0)  # Remove the added batch dimension
        attention_mask = inputs['attention_mask'].squeeze(0)  # Remove the added batch dimension

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [27]:
# For the training dataset
train_dataset = sen_train.drop(['ID'], axis=1)
num_columns = len(train_dataset.columns)
train_dataset['list'] = train_dataset[train_dataset.columns[1:num_columns]].values.tolist()

# Check the length of the feature vectors in the 'list' column
# Assume the length of the feature vectors should be 10
feature_vector_length = 10
train_dataset['list'] = train_dataset['list'].apply(lambda x: x[:feature_vector_length])

train_df = train_dataset[['Tweet', 'list']].copy()  # Select only the 'Tweet' and 'list' columns
train_df = train_df.reset_index(drop=True)  # Reset the index of the DataFrame, dropping the old index

# For the testing dataset
test_dataset = sen_test.drop(['ID'], axis=1)
num_columns = len(test_dataset.columns)
test_dataset['list'] = test_dataset[test_dataset.columns[1:num_columns]].values.tolist()

# Check the length of the feature vectors in the 'list' column
test_dataset['list'] = test_dataset['list'].apply(lambda x: x[:feature_vector_length])

test_df = test_dataset[['Tweet', 'list']].copy()  # Select only the 'Tweet' and 'list' columns
test_df = test_df.reset_index(drop=True)  # Reset the index of the DataFrame, dropping the old index

In [28]:
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [29]:
# Define parameters for training DataLoader
train_params = {'batch_size': TRAIN_BATCH_SIZE,  # Set batch size
                'shuffle': True,  # Shuffle the data
                'num_workers': 0}

# Define parameters for testing DataLoader
test_params = {'batch_size': VALID_BATCH_SIZE,  # Set batch size
                'shuffle': True,  # Shuffle the data
                'num_workers': 0}

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


Creating the Transformer model

In [30]:
from transformers import RobertaModel

class RoBERTaCustom(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

model = RoBERTaCustom(num_classes=10)
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTaCustom(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [31]:
# Define the loss function
def loss_fn(outputs, targets):
    # Use Binary Cross Entropy with Logits Loss for calculating the loss
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

# Define the optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Fine_tuning

In [33]:
def train(epoch):
    # Set the model to training mode
    model.train()
    # Initialize total loss for this epoch
    total_loss = 0

    # Iterate over the training data loader
    for batch_idx, data in enumerate(tqdm(training_loader), 0):
        # Move input tensors to the appropriate device (GPU or CPU)
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['targets'].to(device)

        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Print the shapes of outputs and targets for debugging
        # print("Output shape:", outputs.shape)
        # print("Target shape:", targets.shape)

        # Calculate the batch loss
        loss = loss_fn(outputs, targets)

        # Accumulate the total loss for this epoch
        total_loss += loss.item()

        # Print training progress every 2000 batches
        if batch_idx % 2000 == 0:
            print(f'Iteration: {batch_idx+1}, Epoch: {epoch+1}, Loss: {total_loss/(batch_idx+1)}')

        # Zero the gradients, backward pass, and update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [34]:
EPOCHS = 1 # use 4
for epoch in range(EPOCHS):
    train(epoch)

  0%|          | 0/4500 [00:00<?, ?it/s]

Iteration: 1, Epoch: 1, Loss: 0.6954530477523804


 44%|████▍     | 2000/4500 [2:51:23<3:35:25,  5.17s/it]

Iteration: 2001, Epoch: 1, Loss: 0.3890868563955275


 89%|████████▉ | 4000/4500 [5:42:20<41:57,  5.04s/it]

Iteration: 4001, Epoch: 1, Loss: 0.36619437261913096


100%|██████████| 4500/4500 [6:24:41<00:00,  5.13s/it]


validating the data

In [35]:
def valid():
    model.eval()
    req_targets = []
    req_outputs = []
    valid_loss = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(testing_loader, 0):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            targets = data['targets'].to(device)

            # Forward pass: compute predicted outputs by passing inputs to the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Calculate the batch loss
            loss = loss_fn(outputs, targets)
            valid_loss += loss.item()

            # Append targets and outputs
            req_targets.extend(targets.cpu().detach().numpy().tolist())
            req_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    # Calculate the average validation loss
    valid_loss /= len(testing_loader)

    return req_outputs, req_targets, valid_loss


In [36]:
from sklearn import metrics

outputs, targets, valid_loss = valid()

In [37]:
outputs = np.array(outputs)
targets = np.array(targets)

In [38]:
int_outputs = np.zeros((outputs.shape[0], outputs.shape[1]))

for row in range(outputs.shape[0]):
    for col in range(outputs.shape[1]):
        if outputs[row][col] >= 0.5: int_outputs[row][col] = 1

In [39]:
targets[0], int_outputs[0], outputs[0]

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0.0941153 , 0.00888482, 0.00731148, 0.08180693, 0.06632476,
        0.1539349 , 0.26861301, 0.02606094, 0.01959244, 0.86135316]))

In [40]:
from sklearn.metrics import hamming_loss, jaccard_score, label_ranking_average_precision_score, f1_score

In [41]:
bert_ham_loss = hamming_loss(targets, int_outputs)
bert_jacc_score = jaccard_score(targets, int_outputs, average = 'samples')
bert_lrap = label_ranking_average_precision_score(targets, outputs)
bert_f1_macro = f1_score(targets, int_outputs, average = 'macro')
bert_f1_micro = f1_score(targets, int_outputs, average = 'micro')

In [42]:
print("Test Loss:", valid_loss)
print("Hamming Loss:", bert_ham_loss)
print("Jaccard Score:", bert_jacc_score)
print("Label Ranking Average Precision Score:", bert_lrap)
print("F1 Macro Score:", bert_f1_macro)
print("F1 Micro Score:", bert_f1_micro)

Test Loss: 0.33009273248910903
Hamming Loss: 0.1377
Jaccard Score: 0.44739999999999996
Label Ranking Average Precision Score: 0.7542619444444465
F1 Macro Score: 0.4256093954558824
F1 Micro Score: 0.5186997553303041


In [43]:
torch.save(model, f = '/content/drive/MyDrive/RoBERTa_ft.pth')

# test_part

In [44]:
roberta = torch.load("/content/drive/MyDrive/RoBERTa_ft.pth")
roberta

RoBERTaCustom(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [50]:
gdf = pd.read_csv('/content/drive/MyDrive/guardian_articles_2018_to_2022.csv')
world_news_df = gdf[gdf['sectionName'] == 'World news']
random_world_news = world_news_df['bodyContent'].sample(n=10, random_state=42)

In [77]:

roberta_df = pd.DataFrame(index=range(10))

roberta_df['Tweet'] = random_world_news.reset_index(drop=True)

values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * 10
roberta_df['list'] = values

print(roberta_df)

                                               Tweet  \
0  The death of an 85-year-old man who reportedly...   
1  Josephine Okokon, head of St Martins-in-the-Fi...   
2  Speaking to state governors on Monday, Joe Bid...   
3  Brazilian police have arrested a priest in the...   
4  The US military has confirmed that one of its ...   
5  Covid laws and free mass testing are to be swe...   
6  Brazil’s congress has launched a parliamentary...   
7  Key developments in the global coronavirus out...   
8  More than 1 million people in north-west Engla...   
9  The UK’s military is expected to embark on a n...   

                             list  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
5  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
6  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
7  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
8  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
9  [0, 0, 0, 0, 0, 0, 0

In [78]:
test_dataset = CustomDataset(roberta_df, tokenizer, MAX_LEN)

In [79]:
roberta_test_params = {'batch_size': 1,
                    'shuffle': False,
                    'num_workers': 0
                    }

test_loader = DataLoader(test_dataset, **roberta_test_params)

In [80]:
def test():
    roberta.eval()
    roberta_outputs = []

    with torch.no_grad():
        for unw, data in enumerate(test_loader, 0):
            input_ids = data['input_ids'].to(device, dtype=torch.long)
            attention_mask = data['attention_mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = roberta(input_ids=input_ids, attention_mask=attention_mask)

            roberta_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return roberta_outputs


In [81]:
test_outputs = test()

In [82]:
test_outputs = np.array(test_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [83]:
roberta_df['Optimistic'] = "None"
roberta_df['Thankful'] = "None"
roberta_df['Empathetic'] = "None"
roberta_df['Pessimistic'] = "None"
roberta_df['Anxious'] = "None"
roberta_df['Sad'] = "None"
roberta_df['Annoyed'] = "None"
roberta_df['Denial'] = "None"
roberta_df['Official report'] = "None"
roberta_df['Joking'] = "None"
roberta_df = roberta_df.drop(['list'], axis=1)

In [84]:
for i in range(len(test_outputs)):
    roberta_df.at[i, 'Optimistic'] = test_outputs[i][0]
    roberta_df.at[i, 'Thankful'] = test_outputs[i][1]
    roberta_df.at[i, 'Empathetic'] = test_outputs[i][2]
    roberta_df.at[i, 'Pessimistic'] = test_outputs[i][3]
    roberta_df.at[i, 'Anxious'] = test_outputs[i][4]
    roberta_df.at[i, 'Sad'] = test_outputs[i][5]
    roberta_df.at[i, 'Annoyed'] = test_outputs[i][6]
    roberta_df.at[i, 'Denial'] = test_outputs[i][7]
    roberta_df.at[i, 'Official report'] = test_outputs[i][8]
    roberta_df.at[i, 'Joking'] = test_outputs[i][9]

In [85]:
roberta_df.head()

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,The death of an 85-year-old man who reportedly...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,"Josephine Okokon, head of St Martins-in-the-Fi...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,"Speaking to state governors on Monday, Joe Bid...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Brazilian police have arrested a priest in the...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,The US military has confirmed that one of its ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# roberta_df.to_csv("roberta_final.csv", index = False)